# 모듈 Import

In [32]:
import easydict
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import os
import time
from PIL import ImageFont, ImageDraw, Image

# Best Detector:OpenCV(HOG) + tf_Pose

In [31]:
# TF-Pose, HOG descriptor 객체 생성 
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
cap = cv2.VideoCapture("./Test_video.mp4")

count = 0
while True:
    ret, image = cap.read()
    people_bbox = dict()
    
    if not ret:
        break
    # Bounding Box 그리기
    detected, _ = hog.detectMultiScale(image)
    
    for (x,y,w,h) in detected:    
        # 가로 200pixel 세로 300pixel 이하 제외
        if w < 200 and h < 300: continue
        
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
        
        # model prediction
        input = image[int(y):int(y+h), int(x):int(x+w)]
        model ~~~~
        prediction = ~~~

        people_bbox[(x,y,w,h)] = prediction
        
    # Skeleton 그리기 with Background
    humans = e.inference(image, upsample_size=4.0)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    
    # esc 누르면 종료
    if cv2.waitKey(10) == 27:                    
        break
    print('%d.jpg done' % count)
    count += 1
    
    # Class name print 
    for (x,y,w,h) in people_bbox:
        gesture = people_bbox[(x,y,w,h)]
        cv2.putText(image, gesture, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)
    
    cv2.imshow("Gesture_Recognition", image)

#윈도우 종료
cap.release()
cv2.destroyWindow('Gesture_Recognition')

SyntaxError: invalid syntax (<ipython-input-31-f6fca9eed9e4>, line 25)

# OpenCV(cascade) + tf_Pose

In [ ]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
body_cascade = cv2.CascadeClassifier(
                './opencv-master/opencv-master/data/haarcascades/haarcascade_fullbody.xml')

cap = cv2.VideoCapture(1)
count = 0
while True:
    ret, image = cap.read()
    people_body = []

    if not ret:
        break
    
    # Skeleton 그리기 with Background
    humans = e.inference(image, upsample_size=4.0)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    
    # Bounding Box 그리기
    body = body_cascade.detectMultiScale(image, 1.01, 10)
    for (x,y,w,h) in body:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
        people_body.append(image[int(y):int(y+h), int(x):int(x+w)])
    
    if cv2.waitKey(10) == 27:                    
        break
    print('%d.jpg done' % count)
    count += 1
    
    
    
    cv2.imshow("Gesture_Recognition", image)

#윈도우 종료
cap.release()
cv2.destroyWindow('Gesture_Recognition')

# OpenCV Test with haarcascades

In [23]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
full_body_cascade = cv2.CascadeClassifier(
                './opencv-master/opencv-master/data/haarcascades/haarcascade_fullbody.xml')
lower_body_cascade = cv2.CascadeClassifier(
                './opencv-master/opencv-master/data/haarcascades/haarcascade_lowerbody.xml')
upper_body_cascade = cv2.CascadeClassifier(
                './opencv-master/opencv-master/data/haarcascades/haarcascade_upperbody.xml')

people_body = []
image = cv2.imread('Test.jpg', cv2.IMREAD_COLOR)


# Skeleton 그리기 with Background
humans = e.inference(image, upsample_size=4.0)
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

# Bounding Box 그리기
full_body = full_body_cascade.detectMultiScale(image, 1.01, 10)
lower_body = lower_body_cascade.detectMultiScale(image, 1.01, 10)
upper_body = upper_body_cascade.detectMultiScale(image, 1.01, 10)

for (x,y,w,h) in full_body:
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3) # 빨간색

for (x,y,w,h) in lower_body:
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3) # 초록색

for (x,y,w,h) in upper_body:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3) # 파란색


# if cv2.waitKey(10) == 27:                    
#     break
# print('%d.jpg done' % count)
# count += 1



cv2.imshow("Gesture_Recognition", image)
cv2.imwrite("Test_result_lower_upper.jpg", image)

cv2.waitKey(0)
cv2.destroyAllWindows()

# #윈도우 종료
# cap.release()
# cv2.destroyWindow('Gesture_Recognition')

[2021-01-27 23:34:09,521] [TfPoseEstimator] [INFO] loading graph from C:\Users\pc\Desktop\2021_UGRP\personal\Nwoo\Gesture_Recognition\main_work\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


In [9]:
type(people_body[0])

numpy.ndarray

In [11]:
from PIL import Image

In [13]:
people_body
img = Image.fromarray(people_body[0], 'RGB')
img.show()
# cv2.imshow("Test", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# OpenCV Test with HOG(histogram of Oriented Gradient)

In [23]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

people_bbox = []
people_body = []
image = cv2.imread('Test.jpg', cv2.IMREAD_COLOR)

# Bounding Box 그리기
detected, _ = hog.detectMultiScale(image)
for (x, y, w, h) in detected:
    if w < 200 and h < 300: continue
    cv2.rectangle(image, (x,y),(x+w,y+h), (255,0,0), 3) # 파란색
    people_body.append(image[int(y):int(y+h), int(x):int(x+w)])
    people_bbox.append((x,y,w,h))
    
    

# # Skeleton 그리기 with Background
humans = e.inference(image, upsample_size=4.0)
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)    
    
# Class name print 
for (x,y,w,h) in people_bbox:
    text = "test"
    cv2.putText(image, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)

    
print(people_bbox)
    
cv2.imshow("Gesture_Recognition", image)
# cv2.imwrite("Test_result_hog.jpg", image)

cv2.waitKey(0)
cv2.destroyAllWindows()

# #윈도우 종료
# cap.release()
# cv2.destroyWindow('Gesture_Recognition')

[2021-01-28 13:18:14,590] [TfPoseEstimator] [INFO] loading graph from C:\Users\pc\Desktop\2021_UGRP\personal\Nwoo\Gesture_Recognition\main_work\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


[(474, 262, 423, 846)]


# 데이터셋 생성

In [4]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
# Dataset Path 지정
data_path = ['./1. Original_Datasets/', './2. Jointed_Datasets/', './2. Jointed_Datasets_without_background/']
pose_classes = ['1. Pedestrian/', '2. Lier/', '3. Sitter/', '4. Taxier/', '5. Blocker/']

    
for pose_class in pose_classes:

    original_path = data_path[0]+pose_class
    new_path = data_path[1]+pose_class
    new_path_without_background = data_path[2]+pose_class
    file_list = os.listdir(original_path) # 해당 path 내의 모든 파일 list 가져오기

    for file in file_list:
        # input 단위 설정
        args = easydict.EasyDict({"image": original_path+file})

        # image 로드
        image = cv2.imread(args.image)
        
        # Skeleton 그리기 with Background
        humans = e.inference(image, upsample_size=4.0)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
        
        # 이미지 저장
        cv2.imwrite(new_path+file,image)

        # Skeleton 그리기 without Background
        humans = e.inference(image, upsample_size=4.0)
        image = np.zeros(image.shape,dtype=np.uint8)
        image.fill(255) 
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        # 이미지 저장
        cv2.imwrite(new_path_without_background+file,image)
        
        time.sleep(0.1)
        
    time.sleep(1)

[2021-01-19 16:55:31,874] [TfPoseEstimator] [INFO] loading graph from C:\Users\nakhe\Desktop\UGRP\2021_UGRP\personal\nakon_zoe\3. Side Project\2021.01.25. Gesture Recognition\2021.01.19. Final_Step\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


# 특정 이미지 스켈레톤 뽑기

In [1]:
import easydict
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import os
import time

In [4]:
# 모델 import
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))

# input 단위 설정
args = easydict.EasyDict({"image": 'taxier_test.jpg'})

# image 로드
image = cv2.imread(args.image)

# Skeleton 그리기 with Background
humans = e.inference(image, upsample_size=4.0)
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

# 이미지 저장
cv2.imwrite('taxier_test_back.jpg',image)

# Skeleton 그리기 without Background
humans = e.inference(image, upsample_size=4.0)
image = np.zeros(image.shape,dtype=np.uint8)
image.fill(255) 
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

# 이미지 저장
cv2.imwrite('taxier_test_noback.jpg',image)

[2021-01-19 22:45:29,702] [TfPoseEstimator] [INFO] loading graph from C:\Users\nakhe\Desktop\UGRP\2021_UGRP\personal\nakon_zoe\3. Side Project\2021.01.25. Gesture Recognition\2021.01.19. Final_Step\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


True